## Workshop - Bootstrap

Today we will

1. Show the average unique number of observations when bootstrapping
2. Estimate the standard deviation on the causal effect from a **RANDOMIZED CONTROL TRIAL**

**************************************
# Bootstrap Samples

In one code cell:

- import `numpy` and `numpy.random`
- set the seed to 490
- create *a range* from 0 to 10,000
    - *hint: start with a smaller size to set up the framework*
- create an empty list
- in a 1,000 iteration for loop
    - *hint: start with a smaller size to set up the framework*
    - randomly sample your range your range with replacement with a size equal to the length of your range using `npr.choice()`
    - append your empty list with the length of the the number of unique values from the sampling with replacement
- output the average number of unique values over all bootstrapped samples

In [9]:
import numpy as np
import numpy.random as npr
from tqdm import tqdm

In [47]:
np.random.seed(490)
beta = []
r = np.arange(0, 10000)
for i in tqdm(range(0, 1000)):
    temp = npr.choice(r, len(r))
    beta.append(len(np.unique(temp)))


100%|██████████| 1000/1000 [00:00<00:00, 1016.27it/s]


In [48]:
print('Bootstrap Avg.: %.4f' %  (np.mean(beta) / len(beta)))

Bootstrap Avg.: 6.3216


Is this closer to 1/2, 2/3, or 3/4?

**************
# Randomize Control Trial 

In economics, we call experiments with randomly assigned treatment and control groups __*randomized control trials*__. 
In data science, they are called _**A-B testing**_.

In this application, we will be using a data set from [kaggle](https://www.kaggle.com/samtyagi/audacity-ab-testing). 
We will be using an LPM to estimate the effect of being in a treament group on clicking *something*.
The data is from Audacity, however, there is no information about the experiment specifically. 
We do not know if this is showing different versions of a website, different versions of an advertisement, or something else entirely.



In [15]:
import pandas as pd
from tqdm import tqdm

import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
import os

Load in the audacity data as `ab` with `index_col = timestamp`.
Print the head.

In [27]:
os.chdir('e:\\UIUC\\ECON 490\\DataSet\\')
os.getcwd()
ab = pd.read_csv('homepage_actions.csv', index_col = 'timestamp')


Determine the unique values of `group` and `action`

In [49]:
print(ab['action'].unique())
print(ab['group'].unique())

['view' 'click']
['experiment' 'control']


Create a dummy variable `treatment` for those in the treatment group.
Create a dummy variable `click` for those that clicked.

In [35]:
ab['treatment'] = (ab['group'] == 'experiment')*1
ab['click'] = (ab['action'] == 'click')*1
ab

,id,group,action,is_treatment,is_click,treatment,click
timestamp,,,,,,,
2016-09-24 17:42:27.839496,804196,experiment,view,1,0,1,0
2016-09-24 19:19:03.542569,434745,experiment,view,1,0,1,0
2016-09-24 19:36:00.944135,507599,experiment,view,1,0,1,0
2016-09-24 19:59:02.646620,671993,control,view,0,0,0,0
2016-09-24 20:26:14.466886,536734,experiment,view,1,0,1,0
...,...,...,...,...,...,...,...
2017-01-18 09:11:41.984113,192060,experiment,view,1,0,1,0
2017-01-18 09:42:12.844575,755912,experiment,view,1,0,1,0
2017-01-18 10:01:09.026482,458115,experiment,view,1,0,1,0


Create an object `x` that is the model matrix composed of a constant and the `treatment` variable.
Create an object `y` that is the `click` variable.

In [41]:
# x = pd.DataFrame()
# x['treatment'] = ab['treatment']
# x = sm.add_constant(x)
# y = ab['click']

x = sm.add_constant(ab['treatment'])
y = ab['click']

In one line, fit a statsmodel OLS and print the summary. 
Note the estimate and standard error on the `treatment` variable.

In [43]:
reg = sm.OLS(y, x).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  click   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     3.738
Date:                Tue, 09 Mar 2021   Prob (F-statistic):             0.0532
Time:                        15:07:23   Log-Likelihood:                -4493.7
No. Observations:                8188   AIC:                             8991.
Df Residuals:                    8186   BIC:                             9006.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2186      0.006     34.068      0.000       0.206       0.231
treatment      0.0179      0.009      1.933      0.053      -0.000       0.036
==============================================================================
Omnibus:                     1459.439   Durbin-Watson:                   2.566
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2342.875
Skew:                           1.301   Prob(JB):                         0.00
Kurtosis:                       2.696   Cond. No.                         2.57
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Here we will perform the bootstrap in one code cell.

- set the `npr` seed to 490
- define `n` equal to the number of rows of `ab`
- create an empty list `beta`
- set up a for loop over 2,000 iterations using tqdm
    - use `npr.choice()` to obtain the bootstrap index
    - fit a `LinearRegression()`
        - *hint:* `X` *needs to be a DataFrame, not a Series. Select the* `treatment` *variable using* `ab[['treatment']].iloc[indx]`. `y` *needs to be a Series. Select with only single square brackets.*
    - append the `fit.coef_` to beta
        - *Note: the intercept, which we do not need, is contained seperately in* `fit.intercept_`.

Using one `print()` statment, print the average `beta` with 3 decimal places and the standard deviation of `beta` with 4 decimal places.

Up next, we will produce a histogram. However, we need to perform some preprocessing.

Print the top five observations of `beta` using a slice. Note the format.

To convert to a list we can work with

- use `np.concatenate()` on `beta`
- chain the `.flat` attribute
- wrap the whole thing with `list()`
- overwrite `beta`

Finally, use `matplotlib` to create a histogram of `beta`. 